# JSON-E
Transforms JSON into JSON (with JSON) (and Python, Go, or JS)

In [ ]:
from pathlib import Path
import ipywidgets as W
from wxyz.json_e import JSONE
from wxyz.lab import Editor
from wxyz.core import JSON
from wxyz.lab import DockBox
from wxyz.yaml import YAML
from yaml import safe_dump, safe_load

In [ ]:
THEME = "material"

In [ ]:
default_template = """
message: hello ${key}
k=${num}: true
""".strip()
    
default_context = """
key: world
num: 1
""".strip()

In [ ]:
def make_a_json_e_playground(template_yaml=default_template, context_yaml=default_context):
    template_src = Editor(template_yaml, description="Template", mode="yaml-e", theme=THEME)
    context_src = Editor(context_yaml, description="Context", mode="yaml", theme=THEME)
    output_src = Editor(description="Output", mode="yaml", theme=THEME)

    template = YAML()
    context = YAML()
    jsone = JSONE()

    box = DockBox([
        template_src,
        context_src,
        output_src,
    ], layout=dict(height="40vh"))

    W.jslink((template_src, "value"), (template, "source"))
    W.jslink((context_src, "value"), (context, "source"))
    W.jslink((template, "value"), (jsone, "source"))
    W.jslink((context, "value"), (jsone, "context"))
    W.dlink((jsone, "value"), (output_src, "value"), lambda x: safe_dump(x))
    
    return box

In [ ]:
if __name__ == "__main__":
    pg = make_a_json_e_playground()
    display(pg)

# 😈 Using a Notebook in (@)Context
Linked Data can predictably augment a schema-constrained JSON document. While the resulting document _won't_ conform to the schema, it is suitable as an intermediate format before other activities (e.g. JSON-LD Expansion or SHACL validation) without _requiring_ a reasoner.

- Adds an `@type` to each cell (based on its `cell_type`)
  - this could be done more elgantly with `@context: {"cell_type": "nbformat:definitions/cell_type"}`
- hoists cell `metadata`
  - probably very dangerous: due to ordering, you couldn't _replace_ existing values, but with `$mergeDeep` could prepend to, say, `source` or outputs

In [ ]:
try: 
    default_nb_path = Path(__file__)
except: 
    default_nb_path = Path("JSON-E.ipynb")

In [ ]:
def make_a_jsone_e_notebook_playground(nb_path=default_nb_path):
    box = make_a_json_e_playground()
    context_src.value = safe_dump({"nb": safe_load(nb_path.read_text())})
    template_src.value = """
    $mergeDeep:
      - "@context":
        - gh: https://github.com/
        - nbformat: gh:jupyter/nbformat/blob/master/nbformat/v4/nbformat.v4.schema.json#
      - "@type": nbformat
      - metadata: 
          $eval: nb.metadata
      - nbformat: 
          $eval: nb.nbformat
      - nbformat_minor:
          $eval: nb.nbformat_minor
      - cells:
          $map: 
            $eval: nb.cells
          each(cell):
            $merge:
              - $eval: cell.metadata
              - "@type": nbformat:definitions/${cell.cell_type}_cell
              - $eval: cell
    """
    return box

In [ ]:
if __name__ == "__main__":
    display(make_a_jsone_e_notebook_playground())